<a href="https://colab.research.google.com/github/write-with-neurl/vellum/blob/main/Function_Calling_with_OpenAI_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🖥 Function Calling with OpenAI API

This notebook covers how to:
use the Chat Completions API in combination with external functions to extend:

* Send a simple request to one of OpenAI's LLMs.
* How to define functions for use by these GPT models.
* How to generate parameters from GPT for functions.
* How to call the function when GPT deems conditions are met.

Before we begin, we can run the cell diectly below to wrap output text to a new line once the previous has been filled, which might be useful to view GPT output.

In [ ]:
# Colab users only
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

### 📦 Installation

Before we begin, let's install some dependencies for our demo.

In [ ]:
!pip install --quiet scipy tenacity tiktoken termcolor openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00


Let's now import the necessary packages we need, declare what GPT model we would like to use, import our `api_key`, and instantiate an OpenAI client.

You can select from a series of models listed [here](https://platform.openai.com/docs/models).

Note: explciitly declaring the API key as a string in the notebook is a quick an easy way to use the key, but is not the best practice. Consider defining the `api_key` via an environment variable.

In [ ]:
import json
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
API_KEY = "USE_YOUR_API_KEY_HERE"
client = OpenAI(api_key=API_KEY)

## 📫 Sending Simple Requests

The most basic example on interacting with the API is by sending in a message to talk to GPT.

The main input is an array of message objects in the `messages` parameter, consisting of:

* `role`: Can either be `system`, `user`, or `assistant`.
    * The `assistant` is referencing the Assistant's API, which currently supports three types of tools: Code Interpreter, Retrieval, and Function Calling.
* `content`: String input

Conversations can be as short as one message or many back and forth turns.

Typically, a conversation is formatted with a system message first, followed by alternating user and assistant messages.

To learn more about these requests, click [here](https://platform.openai.com/docs/guides/text-generation/chat-completions-api).

In [ ]:
completion = client.chat.completions.create(
  model=GPT_MODEL,
  messages=[
    {"role": "user", "content": "What's machine learning?"}
  ]
)
print(completion.choices[0].message.content)

Machine learning is a field of artificial intelligence (AI) that focuses on enabling computers to learn and improve from experience without being explicitly programmed. It involves the development of algorithms and models that allow machines to analyze and make predictions or decisions based on data patterns, rather than being explicitly programmed for a specific task. Through training on large sets of data, machine learning algorithms can identify and learn from patterns, and then apply this knowledge to make accurate predictions or take actions in new and unseen situations. Machine learning is used in various domains like image and speech recognition, natural language processing, recommendation systems, and many others.


As mentioned, you can define what the system's role is by modifying the `role`.

In [ ]:
completion = client.chat.completions.create(
  model=GPT_MODEL,
  messages=[
    {"role": "system", "content": "You are a poetic haiku assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "What's machine learning?"}
  ]
)

print(completion.choices[0].message.content)

In code's clever dance,
Machine learns from each nuance,
Wisdom amplified.


Instead of waiting for responses to be completely generated, they can also be streamed while being generated.

In [ ]:
stream = client.chat.completions.create(
    model=GPT_MODEL,
    messages=[{"role": "user", "content": "What is machine learning?"}],
    stream=True,
)
for chunk in stream:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

Machine learning is a subset of artificial intelligence that enables computers to learn and make decisions without being explicitly programmed. It is based on the idea that systems can learn from data, identify patterns, and make predictions or decisions with minimal human intervention. Machine learning algorithms analyze large amounts of data to find patterns, improve performance, and make accurate predictions by continuously adapting and learning from new data. It is used in various applications such as image and speech recognition, autonomous vehicles, recommendation systems, fraud detection, and many more.

On top of this, if we have a `messages` list to keep track of our current conversation, we can hold a back and forth conversation with GPT.

We will define a function to request a response to keep our sample clear and straightforward.

In [ ]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


def create_message(role, content):
    return {'role': role, 'content': content}

Now, we can start a sample back and forth conversation.

In [ ]:
# Let's store our previous message from above.
messages = [
    {"role": "user", "content": "What is machine learning?"}
]
print('User:\n', messages[0]['content'], '\n')

# Get a response, add it to our list of messages (our conversation)
response = chat_completion_request(messages)
message = response.choices[0].message.content
print('Response from GPT:\n', message, '\n')
messages.append(create_message('system', message))

# Let's ask GPT to summarize its previous response.
# We'll add our request to the messages list before asking for another response.
user_req = create_message('user', 'Can you summarize what you just told me in one sentence?')
messages.append(user_req)
print('User:\n', user_req['content'], '\n')

# Send the response, print the output.
response = chat_completion_request(messages)
print('Response from GPT:\n', response.choices[0].message.content, '\n')


User:
 What is machine learning? 

Response from GPT:
 Machine learning is a subset of artificial intelligence (AI) that focuses on the development of algorithms and models that enable computers to learn and make predictions or decisions without being explicitly programmed. It involves the use of statistical techniques to enable machines to automatically learn and improve from experience or data, without being explicitly programmed for every specific task. By using patterns and inference, machine learning algorithms can identify and understand complex patterns and relationships within data, and make predictions or decisions based on those patterns. Machine learning finds applications in various industries, including areas like healthcare, finance, marketing, and many more. 

User:
 Can you summarize what you just told me in one sentence? 

Response from GPT:
 Machine learning is a branch of artificial intelligence that enables computers to learn from data, make predictions or decisions

## 🔔 Function Calling

Function calling is a part of OpenAI's Assistants API. As mentioned previously, there are types of assistants at this current time of writing: Code Interpreter, Retrieval, and Function calling.

Of course, in this demo, we'll focus on Function Calling.
* Function calling is the ability of the LLM to perform a specific task by returning a deterministic and structured output.

* In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions (depending on if the model supports multiple parallel calls, such a s GPT4).

* The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

For more information on the Assistants API, click [here](https://platform.openai.com/docs/assistants/overview?context=with-streaming).

### Helper Functions

We'll establish one more helper function to print out all of our messages now. For the demo below, we also require `chat_completion_request()` and `create_message()` from our `Sending Simple Requests` examples above.

In [ ]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


### ⏳ Generating Function Arguments

Before generating our function arguments, we need to define what `tools` our OpenAI `client()` has access to. `tools` are a list of `dict` items that indicate what type of tool it is (Code Interpreter, Retrieval, and Function Calling), as well as other information describing that tool.

Below, we initialize `function` tools for Function Calling. However, you can have any mix of `tools` that you deem fit.

We initialize `get_current_weather()` and `get_n_day_weather_forecast()` with its necessary parameters.

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

Now, we can begin sending requests to GPT with our `tools`.

If we prompt the model about the current weather, it will respond with some clarifying questions (as shown in `content` in the response).

In [ ]:
# Define messages
messages = []
messages.append(
    create_message(
        "system",
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
)
messages.append(create_message("user", "What's the weather like today?"))

# Submit response
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure, could you please provide me with your current location?', role='assistant', function_call=None, tool_calls=None))


Once we provide the missing information, it will generate the appropriate function arguments for us.

Notice how the `finish_reason` is now `tool_calls`, indicating some `tool` defined in our `tools` schema is being called.

In [ ]:
# Define messages
messages.append(create_message("user", "I'm in San Francisco, CA"))
chat_response = chat_completion_request(
    messages, tools=tools
)

# Submit response
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_npQlZt0Ef84rYiT6Dat8V1xO', function=Function(arguments='{\n  "location": "San Francisco, CA",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')]))


By prompting it differently, we can get it to target the other function we've told it about.

In [ ]:
# Define messages
messages = []
messages.append(
    create_message(
        "system",
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
)
messages.append(create_message("user", "what is the weather going to be like in San Francisco, CA over the next x days"))

# Submit response
chat_response = chat_completion_request(
    messages, tools=tools
)
messages.append(chat_response.choices[0].message)
print(chat_response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Sure, I can help you with that. Please provide me with the number of days you would like to forecast for San Francisco, CA.', role='assistant', function_call=None, tool_calls=None))


Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [ ]:
messages.append(create_message("user", "5 days"))
chat_response = chat_completion_request(
    messages, tools=tools
)
print(chat_response.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WBBLCK4M3NL9wlR81bhqM2KB', function=Function(arguments='{\n  "location": "San Francisco, CA",\n  "format": "celsius",\n  "num_days": 5\n}', name='get_n_day_weather_forecast'), type='function')]))


#### Forcing the use of specific functions or no function

We can force the model to use a specific function, for example get_n_day_weather_forecast by using the function_call argument. By doing so, we force the model to make assumptions about how to use it.

In [ ]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
)
print(chat_response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_FQSl6U46sIG2HyXHQ1UnNMrm', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}', name='get_n_day_weather_forecast'), type='function')]))


In [ ]:
# if we don't force the model to use get_n_day_weather_forecast it may not
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools
)
print(chat_response.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_I8NDXwfjmw3QmclV2pfZgBTZ', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}', name='get_current_weather'), type='function')]))


We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
print(chat_response.choices[0])


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{ "location": "Toronto, Canada", "format": "celsius" }', role='assistant', function_call=None, tool_calls=None))


#### Parallel Function Calling

Newer models like gpt-4-1106-preview or gpt-3.5-turbo-1106 can call multiple functions in one turn.

In [ ]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Glasgow over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model='gpt-3.5-turbo-1106'
)

assistant_message = chat_response.choices[0].message.tool_calls
print(assistant_message)

[ChatCompletionMessageToolCall(id='call_oEWfcqY5wiBNAGw8Rb6xlymf', function=Function(arguments='{"location": "San Francisco, CA", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function'), ChatCompletionMessageToolCall(id='call_yBIdc8jb2m4c3Z2zB4NUEofO', function=Function(arguments='{"location": "Glasgow", "format": "celsius", "num_days": 4}', name='get_n_day_weather_forecast'), type='function')]


### 📞 Calling Functions from Generated Inputs
Next, we'll implement some of these functions to return some response, showing how we'd "link" input from our LLM output to our code logic we'd write in a traditional development environment.

For our "linking", we'll have a function that will check for the string of our returns output prompt from the LLM, and call the appropriate function based on that message.

In [ ]:
import json

def get_current_weather(location, format):
    return "Call successful from get_current_weather()."


def get_n_day_weather_forecast(location, format, num_days):
    return "Call successful from get_n_day_weather_forecast()"


def execute_function_call(message):
    args = json.loads(msg.tool_calls[0].function.arguments)
    if message.tool_calls[0].function.name == "get_current_weather":
        results = get_current_weather(args["location"], args["format"])
    elif message.tool_calls[0].function.name == "get_n_day_weather_forecast":
        results = get_n_day_weather_forecast(args["location"], args["format"], args["num_days"])
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

Now, let's try to generate some our function arguments once more, but pipe this to call `execute_function_call()`, which will call other functions we've implemented based on the function name in `message`.

In [ ]:
# Define messages
messages = []
messages.append(
    create_message(
        "system",
        "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
    )
)
messages.append(create_message("user", "what is the weather going to be like in San Francisco, CA?"))

# Submit response
chat_response = chat_completion_request(messages, tools)

# Parse response
msg = chat_response.choices[0].message
messages.append({"role": msg.role, "content": msg.tool_calls[0].function})
msg_func = str(msg.tool_calls[0].function)

# Call corresponding function
if msg.tool_calls:
    results = execute_function_call(msg)
    messages.append({"role": "function",
                     "tool_call_id": msg.tool_calls[0].id,
                     "name": msg.tool_calls[0].function.name,
                     "content": results
                     })
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: what is the weather going to be like in San Francisco, CA?

assistant: Function(arguments='{\n  "location": "San Francisco, CA",\n  "format": "celsius"\n}', name='get_current_weather')

function (get_current_weather): Call successful from get_current_weather().



# 🎉 Congrats!

With the demo above, you should be able to start development with LLMs incorporated into your application.

To learn how to attach knowledge bases for knowledge retrieval, click [here](https://cookbook.openai.com/examples/how_to_call_functions_for_knowledge_retrieval)